# Chapter 10

This chapter is going to introduce three complex types of data that are extremely important for your smart contracts.


So far, the smart contracts we have written were quite simple: they received data as input, applied some logic to the data and output a result. However, this is not what most contracts do. Smart contracts are an interesting solution to many problems because they can retain information and the new input sent to the contract can be compared and/or used against the data that's already in the contract!

In a previous chapter, we introduced the `list` type. Lists allow us to store multiple values of the same type in an ordered fashion. However, they lack some features that you may need for your smart contract. For example, it is not possible natively to know if a list contains a certain value (i.e without implementing a loop).

Michelson provides three other types of values that can store other values: sets, maps and big maps. Their different properties will be explained below but in a nutshell, sets are lists of unique elements, maps are tables where each value matches a unique key and big maps are maps containing a large number of values. Let's start with sets!

## Working with sets

Sets are a sorted collection of unique values. A `set` is like a `list` with the main difference of storing only unique values in a definite order. While `{ 1 ; 2 ; 1 ; 2 ; 1 ; 2 }` is a valid value of type `(list int)`, it wouldn't be a valid value of type `(set int)`. Here is how to create an empty set:

In [1]:
storage (set int) ;
parameter unit ;
code {
    DROP ;
    EMPTY_SET int ;
    NIL operation ;
    PAIR ;
} ;

RUN %default Unit { 1 };

storage: updated
parameter: updated
code: updated
BEGIN %default / _ => (Unit * {1})
DROP / (Unit * {1}) => _
EMPTY_SET / _ => {}
NIL / _ => []
PAIR / [] : {} => ([] * {})
END %default / ([] * {}) => _

type,value
set,[]


You can use the **`EMPTY_SET`** instruction to create an empty set. It takes 1 argument, the type of the elements you will store in the set. It must be a comparable type: you can, for example, put strings or integers in a set but you cannot put a map or a set inside a set.

Next, you want to save some data inside the set. You can use the **`UPDATE`** instruction to push new values in the set. This instruction requires a little setup illustrated in the example below:

In [2]:
storage (set int) ;
parameter unit ;
code {
    DROP ;
    EMPTY_SET int ;
    PUSH bool True;
    PUSH int 5 ;
    UPDATE ;
    NIL operation ;
    PAIR ;
} ;

RUN %default Unit { 1 };

storage: updated
parameter: updated
code: updated
BEGIN %default / _ => (Unit * {1})
DROP / (Unit * {1}) => _
EMPTY_SET / _ => {}
PUSH / _ => True
PUSH / _ => 5
UPDATE / 5 : True : {} => {5}
NIL / _ => []
PAIR / [] : {5} => ([] * {5})
END %default / ([] * {5}) => _

type,value
set (int),[5]


We could also have used the set in the storage:

In [3]:
storage (set int) ;
parameter unit ;
code {
    CDR ;
    PUSH bool True;
    PUSH int 5 ;
    UPDATE ;
    NIL operation ;
    PAIR ;
} ;

RUN %default Unit { 1 };

storage: updated
parameter: updated
code: updated
BEGIN %default / _ => (Unit * {1})
CDR / (Unit * {1}) => {1}
PUSH / _ => True
PUSH / _ => 5
UPDATE / 5 : True : {1} => {1, 5}
NIL / _ => []
PAIR / [] : {1, 5} => ([] * {1, 5})
END %default / ([] * {1, 5}) => _

type,value
set (int),"[1, 5]"


Here is what happens in the stack: the **`UPDATE`** instruction requires 3 elements in the stack before proceeding: 
1. The element to add to the set
2. A boolean value set to `True` to indicate that you want to add a new element to the set
3. The set in which you want to store the new value

If the element is already present in the stack, the execution will just continue without any change. Otherwise, the new element will be pushed at the head position of the set.

If you set the boolean value to `False` and provide a value that already exists in the set, **`UPDATE`** will remove this element from the set:

In [4]:
storage (set int) ;
parameter unit ;
code {
    CDR ;
    PUSH bool False;
    PUSH int 5 ;
    UPDATE ;
    NIL operation ;
    PAIR ;
} ;

RUN %default Unit { 1 ; 2 ; 3 ; 4 ; 5 };

storage: updated
parameter: updated
code: updated
BEGIN %default / _ => (Unit * {1, 2, 3, 4, 5})
CDR / (Unit * {1, 2, 3, 4, 5}) => {1, 2, 3, 4, 5}
PUSH / _ => False
PUSH / _ => 5
UPDATE / 5 : False : {1, 2, 3, 4, 5} => {1, 2, 3, 4}
NIL / _ => []
PAIR / [] : {1, 2, 3, 4} => ([] * {1, 2, 3, 4})
END %default / ([] * {1, 2, 3, 4}) => _

type,value
set (int),"[1, 2, 3, 4]"


Now you understand why you must set up correctly the stack before using **`UPDATE`**: the 3 required elements must be in the right order and the boolean element must be set to the right value in order to add or remove a value from the set. Here is a more complex example:

In [6]:
storage (set string) ;
parameter (pair string string) ;
code {
    UNPPAIAIR ;
    DIP { SWAP } ;
    PUSH bool True ;
    SWAP ;
    UPDATE ;
    SWAP ;
    PUSH bool False ;
    SWAP ;
    UPDATE ;
    NIL operation ;
    PAIR ;
} ;

RUN %default (Pair "mango" "apple") { "apple" ; "banana" ; "strawberry" };

storage: updated
parameter: updated
code: updated
BEGIN %default / _ => (('mango' * 'apple') * {'apple', 'banana', 'strawberry'})
UNPAIR / (('mango' * 'apple') * {'apple', 'banana', 'strawberry'}) => ('mango' * 'apple') : {'apple', 'banana', 'strawberry'}
UNPAIR / ('mango' * 'apple') => 'mango' : 'apple'
DIP / * => _
SWAP / 'apple' : {'apple', 'banana', 'strawberry'} => {'apple', 'banana', 'strawberry'} : 'apple'
DIP 1 / _ => *
PUSH / _ => True
SWAP / True : 'mango' => 'mango' : True
UPDATE / 'mango' : True : {'apple', 'banana', 'strawberry'} => {'apple', 'banana', 'mango', 'strawberry'}
SWAP / {'apple', 'banana', 'mango', 'strawberry'} : 'apple' => 'apple' : {'apple', 'banana', 'mango', 'strawberry'}
PUSH / _ => False
SWAP / False : 'apple' => 'apple' : False
UPDATE / 'apple' : False : {'apple', 'banana', 'mango', 'strawberry'} => {'banana', 'mango', 'strawberry'}
NIL / _ => []
PAIR / [] : {'banana', 'mango', 'strawberry'} => ([] * {'banana', 'mango', 'strawberry'})
END %default / ([]

type,value
set (string),"['banana', 'mango', 'strawberry']"


First, we deconstruct the parameter with **`UNPPAIAIR`** to get our two fruits on the stack with the set of fruits below. Then, we push *mango* into the set of fruits before removing *apple*. A few **`SWAP`** instructions are necessary to put all the elements in the right order. Observe how the boolean values are set to `True` and `False` according to the effect you want to create on the set.

Before trying to remove or add a value to a set, it would be great to check if the value is already inside. This is what you can achieve with the **`MEM`** instruction:

In [7]:
storage (set string) ;
parameter string ;
code {
    UNPAIR ;
    DIP { DUP } ;
    DUP ;
    DIP { SWAP } ;
    MEM ;
    IF { FAIL } { PUSH bool True ; SWAP ; UPDATE } ;
    NIL operation ;
    PAIR ;
} ;

RUN %default "mango" { "apple" ; "banana" ; "strawberry" };

storage: updated
parameter: updated
code: updated
BEGIN %default / _ => ('mango' * {'apple', 'banana', 'strawberry'})
UNPAIR / ('mango' * {'apple', 'banana', 'strawberry'}) => 'mango' : {'apple', 'banana', 'strawberry'}
DIP / * => _
DUP / {'apple', 'banana', 'strawberry'} => {'apple', 'banana', 'strawberry'} : {'apple', 'banana', 'strawberry'}
DIP 1 / _ => *
DUP / 'mango' => 'mango' : 'mango'
DIP / * => _
SWAP / 'mango' : {'apple', 'banana', 'strawberry'} => {'apple', 'banana', 'strawberry'} : 'mango'
DIP 1 / _ => *
MEM / 'mango' : {'apple', 'banana', 'strawberry'} => False
IF / False => _
PUSH / _ => True
SWAP / True : 'mango' => 'mango' : True
UPDATE / 'mango' : True : {'apple', 'banana', 'strawberry'} => {'apple', 'banana', 'mango', 'strawberry'}
NIL / _ => []
PAIR / [] : {'apple', 'banana', 'mango', 'strawberry'} => ([] * {'apple', 'banana', 'mango', 'strawberry'})
END %default / ([] * {'apple', 'banana', 'mango', 'strawberry'}) => _

type,value
set (string),"['apple', 'banana', 'mango', 'strawberry']"


In this contract, we duplicate the values passed in the parameter because **`MEM`** is going to remove the element and the set we want to test. If the value is not in the set, it will be added, otherwise, the contract will fail. As expected, *mango* is not in the set, so the value gets added to the final set. If you replace *mango* with *apple*, you will see the contract fail.

As it is also the case for other types storing multiple values, you can check the size of a set by using the **`SIZE`** instruction:

In [8]:
storage nat ;
parameter (set string) ;
code {
    CAR ;
    SIZE ;
    NIL operation ;
    PAIR ;
} ;

RUN %default { "apple" ; "banana" ; "strawberry" } 0;

storage: updated
parameter: updated
code: updated
BEGIN %default / _ => ({'apple', 'banana', 'strawberry'} * 0)
CAR / ({'apple', 'banana', 'strawberry'} * 0) => {'apple', 'banana', 'strawberry'}
SIZE / {'apple', 'banana', 'strawberry'} => 3
NIL / _ => []
PAIR / [] : 3 => ([] * 3)
END %default / ([] * 3) => _

type,value
nat,3


The **`SIZE`** instruction returns a value of type `nat`. Add more strings to the set to see it working!

## Working with maps

Maps and big maps are probably the type of complex values you will work with most of the time. As big maps are a kind of map, we start with maps!

Maps provide a very convenient way of storing data: they are like a table with two columns, a simple value on the left side that you can use to retrieve the data on the right side. This allows to store complex data that you can find very quickly and easily. For example, in a token contract, you can associate an address with its balance and the chosen allowances. When creating a new map, you have to specifiy the type of the keys and the type of the values, as you won't be able to store data that are not of the specified type. Here is how to create an empty map:

In [9]:
storage (map address nat) ;
parameter unit ;
code {
    DROP ;
    EMPTY_MAP address nat ;
    NIL operation ;
    PAIR ;
} ;

RUN %default Unit {} ;

storage: updated
parameter: updated
code: updated
BEGIN %default / _ => (Unit * {})
DROP / (Unit * {}) => _
EMPTY_MAP / _ => {}
NIL / _ => []
PAIR / [] : {} => ([] * {})
END %default / ([] * {}) => _

type,value
map (address nat),{}


You can use any type you want as a key as long as it is a comparable type. Note also that map keys are lexicographically sorted.

Adding and removing values from a map is going to be a little easier than doing it with a set 😅 However, we are going to use the same instruction, **`UPDATE`**:

In [10]:
storage (map address nat) ;
parameter unit ;
code {
    DROP ;
    EMPTY_MAP address nat ;
    PUSH (option nat) (Some 5) ;
    PUSH address "tz1VSUr8wwNhLAzempoch5d6hLRiTh8Cjcjb" ;
    UPDATE ;
    NIL operation ;
    PAIR ;
} ;

RUN %default Unit {} ;

storage: updated
parameter: updated
code: updated
BEGIN %default / _ => (Unit * {})
DROP / (Unit * {}) => _
EMPTY_MAP / _ => {}
PUSH / _ => 5?
PUSH / _ => tz1VSU…cjb
UPDATE / tz1VSU…cjb : 5? : {} => {tz1VSU…cjb: 5}
NIL / _ => []
PAIR / [] : {tz1VSU…cjb: 5} => ([] * {tz1VSU…cjb: 5})
END %default / ([] * {tz1VSU…cjb: 5}) => _

type,value
map (address nat),{'tz1VSUr8wwNhLAzempoch5d6hLRiTh8Cjcjb': 5}


In order to add a new key/pair value to a map, the stack must have three elements in the following order:
1. The key associated to the value to add
2. The value you want to add
3. The map that you want to update  

When the stack is correctly set up, you can call the **`UPDATE`** instruction. It is important that the value you use to update the map is an `optional` value with an argument whose type is going to be the expected type of the values in the map. If you use `(Some type)`, you will add a new value to the map. However, if you use `(None)`, you will remove the value at the provided key:

In [12]:
storage (map address nat) ;
parameter unit ;
code {
    CDR ;
    PUSH (option nat) (None) ;
    PUSH address "tz1VSUr8wwNhLAzempoch5d6hLRiTh8Cjcjb" ;
    UPDATE ;
    NIL operation ;
    PAIR ;
} ;

RUN %default Unit { Elt "tz1UVzNHTxMzkPn1uMXaSCTJYBQQc4x5dyNE" 10 ; Elt "tz1VSUr8wwNhLAzempoch5d6hLRiTh8Cjcjb" 5 } ;

storage: updated
parameter: updated
code: updated
BEGIN %default / _ => (Unit * {tz1UVz…yNE: 10, tz1VSU…cjb: 5})
CDR / (Unit * {tz1UVz…yNE: 10, tz1VSU…cjb: 5}) => {tz1UVz…yNE: 10, tz1VSU…cjb: 5}
PUSH / _ => None
PUSH / _ => tz1VSU…cjb
UPDATE / tz1VSU…cjb : None : {tz1UVz…yNE: 10, tz1VSU…cjb: 5} => {tz1UVz…yNE: 10}
NIL / _ => []
PAIR / [] : {tz1UVz…yNE: 10} => ([] * {tz1UVz…yNE: 10})
END %default / ([] * {tz1UVz…yNE: 10}) => _

type,value
map (address nat),{'tz1UVzNHTxMzkPn1uMXaSCTJYBQQc4x5dyNE': 10}


As you can see from this example, giving a `(None)` value to the map value before calling **`UPDATE`** will remove the corresponding key/value pair. If the key doesn't exist, this will have no effect on the map or the contract. Because of this, you may want to check first if a key exists in a map before removing it or updating it. This is what the **`MEM`** instruction is for:

In [14]:
storage bool ;
parameter (map address nat) ;
code {
    CAR ;
    PUSH address "tz1VSUr8wwNhLAzempoch5d6hLRiTh8Cjcjb" ;
    MEM ;
    NIL operation ;
    PAIR ;
} ;

RUN %default { Elt "tz1UVzNHTxMzkPn1uMXaSCTJYBQQc4x5dyNE" 10 ; Elt "tz1VSUr8wwNhLAzempoch5d6hLRiTh8Cjcjb" 5 } False ;

storage: updated
parameter: updated
code: updated
BEGIN %default / _ => ({tz1UVz…yNE: 10, tz1VSU…cjb: 5} * False)
CAR / ({tz1UVz…yNE: 10, tz1VSU…cjb: 5} * False) => {tz1UVz…yNE: 10, tz1VSU…cjb: 5}
PUSH / _ => tz1VSU…cjb
MEM / tz1VSU…cjb : {tz1UVz…yNE: 10, tz1VSU…cjb: 5} => True
NIL / _ => []
PAIR / [] : True => ([] * True)
END %default / ([] * True) => _

type,value
bool,True


This very simple contract checks if a single address is present in the map passed as a parameter. Please remember that **`MEM`** is going to pop the key and the map from the stack, so you should duplicate them if you want to use them later:

In [16]:
storage (map address nat) ;
parameter address ;
code {
    DUP ;
    DIP { UNPAIR } ;
    UNPAIR ;
    MEM ;
    IF
        { PUSH (option nat) (None) ; SWAP ; UPDATE } ## Removes key/value pair if exists
        { PUSH (option nat) (Some 1) ; SWAP ; UPDATE } ; ## Adds key/value pair if doesn't exist
    NIL operation ;
    PAIR ;
} ;

RUN %default "tz1NhNv9g7rtcjyNsH8Zqu79giY5aTqDDrzB" { Elt "tz1UVzNHTxMzkPn1uMXaSCTJYBQQc4x5dyNE" 10 ; Elt "tz1VSUr8wwNhLAzempoch5d6hLRiTh8Cjcjb" 5 } ;

storage: updated
parameter: updated
code: updated
BEGIN %default / _ => (tz1NhN…rzB * {tz1UVz…yNE: 10, tz1VSU…cjb: 5})
DUP / (tz1NhN…rzB * {tz1UVz…yNE: 10, tz1VSU…cjb: 5}) => (tz1NhN…rzB * {tz1UVz…yNE: 10, tz1VSU…cjb: 5}) : (tz1NhN…rzB * {tz1UVz…yNE: 10, tz1VSU…cjb: 5})
DIP / * => _
UNPAIR / (tz1NhN…rzB * {tz1UVz…yNE: 10, tz1VSU…cjb: 5}) => tz1NhN…rzB : {tz1UVz…yNE: 10, tz1VSU…cjb: 5}
DIP 1 / _ => *
UNPAIR / (tz1NhN…rzB * {tz1UVz…yNE: 10, tz1VSU…cjb: 5}) => tz1NhN…rzB : {tz1UVz…yNE: 10, tz1VSU…cjb: 5}
MEM / tz1NhN…rzB : {tz1UVz…yNE: 10, tz1VSU…cjb: 5} => False
IF / False => _
PUSH / _ => 1?
SWAP / 1? : tz1NhN…rzB => tz1NhN…rzB : 1?
UPDATE / tz1NhN…rzB : 1? : {tz1UVz…yNE: 10, tz1VSU…cjb: 5} => {tz1NhN…rzB: 1, tz1UVz…yNE: 10, tz1VSU…cjb: 5}
NIL / _ => []
PAIR / [] : {tz1NhN…rzB: 1, tz1UVz…yNE: 10, tz1VSU…cjb: 5} => ([] * {tz1NhN…rzB: 1, tz1UVz…yNE: 10, tz1VSU…cjb: 5})
END %default / ([] * {tz1NhN…rzB: 1, tz1UVz…yNE: 10, tz1VSU…cjb: 5}) => _

type,value
map (address nat),"{'tz1NhNv9g7rtcjyNsH8Zqu79giY5aTqDDrzB': 1, 'tz1UVzNHTxMzkPn1uMXaSCTJYBQQc4x5dyNE': 10, 'tz1VSUr8wwNhLAzempoch5d6hLRiTh8Cjcjb': 5}"


The provided address was not a value in the map, so the contract added it and gave it a balance of `1`. Now, if you change the address in the parameter for an address that's already in the map, you will see it disappear in the final map!

After checking if a key exists in a map, you probably want to get the value that's associated with it! Michelson provides the **`GET`** instruction to retrieve values bound to a key in a map:

In [19]:
storage string ;
parameter (pair string (map string string)) ;
code {
    CAR ;
    UNPAIR ;
    GET ;
    IF_NONE
        { PUSH string "none" }
        {} ;
    NIL operation ;
    PAIR ;
} ;

RUN %default (Pair "banana" { Elt "apple" "green" ; Elt "banana" "yellow" ; Elt "cherry" "red" }) "" ;

storage: updated
parameter: updated
code: updated
BEGIN %default / _ => (('banana' * {'apple': 'green', 'banana': 'yellow', 'cherry': 'red'}) * '')
CAR / (('banana' * {'apple': 'green', 'banana': 'yellow', 'cherry': 'red'}) * '') => ('banana' * {'apple': 'green', 'banana': 'yellow', 'cherry': 'red'})
UNPAIR / ('banana' * {'apple': 'green', 'banana': 'yellow', 'cherry': 'red'}) => 'banana' : {'apple': 'green', 'banana': 'yellow', 'cherry': 'red'}
GET / 'banana' : {'apple': 'green', 'banana': 'yellow', 'cherry': 'red'} => 'yellow'?
IF_NONE / 'yellow'? => 'yellow'
NIL / _ => []
PAIR / [] : 'yellow' => ([] * 'yellow')
END %default / ([] * 'yellow') => _

type,value
string,yellow


This very simple code sends a string and a map to the contract that will check if the string is a key in the map. If it is, it will save the color of the fruit in the storage. If it is not, it will save `"none"` in the storage instead. We could also have gone a different road and used **`IF_SOME`** to check if the **`GET`** instruction returns anything:

In [20]:
storage string ;
parameter (pair string (map string string)) ;
code {
    CAR ;
    UNPAIR ;
    GET ;
    IF_SOME
        {} 
        { PUSH string "none" };
    NIL operation ;
    PAIR ;
} ;

RUN %default (Pair "banana" { Elt "apple" "green" ; Elt "banana" "yellow" ; Elt "cherry" "red" }) "" ;

storage: updated
parameter: updated
code: updated
BEGIN %default / _ => (('banana' * {'apple': 'green', 'banana': 'yellow', 'cherry': 'red'}) * '')
CAR / (('banana' * {'apple': 'green', 'banana': 'yellow', 'cherry': 'red'}) * '') => ('banana' * {'apple': 'green', 'banana': 'yellow', 'cherry': 'red'})
UNPAIR / ('banana' * {'apple': 'green', 'banana': 'yellow', 'cherry': 'red'}) => 'banana' : {'apple': 'green', 'banana': 'yellow', 'cherry': 'red'}
GET / 'banana' : {'apple': 'green', 'banana': 'yellow', 'cherry': 'red'} => 'yellow'?
IF_NONE / 'yellow'? => 'yellow'
NIL / _ => []
PAIR / [] : 'yellow' => ([] * 'yellow')
END %default / ([] * 'yellow') => _

type,value
string,yellow


Whether you use **`IF_NONE`** or **`IF_SOME`**, if a value is found, it will be unwrapped from the optional value and dumped onto the stack. In this example, the values in the map are of type `string`, so **`GET`** returns `(Some string)` and **`IF_NONE`**/**`IF_SOME`** unwraps it and brings `string` to the stack.

There is a last instruction to check before ending our tour of maps. Michelson provides an instruction to know the size of a given map, you guessed it, **`SIZE`**!

In [21]:
storage nat ;
parameter (map string nat) ;
code {
    CAR ;
    SIZE ;
    NIL operation ;
    PAIR ;
} ;

RUN %default { Elt "apple" 32 ; Elt "banana" 24 ; Elt "cherry" 16 } 0 ;

storage: updated
parameter: updated
code: updated
BEGIN %default / _ => ({'apple': 32, 'banana': 24, 'cherry': 16} * 0)
CAR / ({'apple': 32, 'banana': 24, 'cherry': 16} * 0) => {'apple': 32, 'banana': 24, 'cherry': 16}
SIZE / {'apple': 32, 'banana': 24, 'cherry': 16} => 3
NIL / _ => []
PAIR / [] : 3 => ([] * 3)
END %default / ([] * 3) => _

type,value
nat,3


As usual, this kind of instruction returns a `nat` value.

Now that you have a better understanding of `maps` in Michelson, we can check `big maps`. Under the hood, big maps are actually maps. The only major difference is that when Michelson loads a map into memory, it goes through all the key/value pairs and get them ready to be used by your code. This consumes gas and can become very expensive if the map is huge. In the case of a big map, Michelson does not prepare the map and will only access the key/value pair you request. This will be a lot cheaper than using a map with a big drawback though: you won't get any information about the map outside of the key/value pair you are trying to access. This means instructions available on `map` like **`SIZE`**, as well as iterative instructions like **`ITER`** and **`MAP`**, are not possible on `big_map` (more on that in the next chapter). The only instructions you can use with big maps are **`EMPTY_BIG_MAP`**, **`GET`**, **`MEM`** and **`UPDATE`**. Let's try to create an example that uses all the available instructions for big maps!

In [22]:
storage (big_map string nat) ;
parameter unit ;
code {
    DROP ;
    EMPTY_BIG_MAP string nat ;
    PUSH nat 15 ;
    SOME ;
    PUSH string "cherry" ;
    UPDATE ;
    PUSH nat 22 ;
    SOME ;
    PUSH string "banana" ;
    UPDATE ;
    DUP ;
    PUSH string "cherry" ;
    DUP ;
    SWAP ;
    DIP { SWAP } ;
    MEM ;
    IF
        { 
            DIP { DUP } ;
            DUP ;
            DIP { SWAP } ;
            GET ;
            IF_SOME
                { PUSH nat 5 ; ADD ; SOME ; SWAP ; UPDATE }
                { DROP }
        }
        { DROP } ;
    NIL operation ;
    PAIR ;
};

RUN %default Unit {} ;

storage: updated
parameter: updated
code: updated
BEGIN %default / _ => (Unit * <-1>)
DROP / (Unit * <-1>) => _
EMPTY_BIG_MAP / _ => <-2>
PUSH / _ => 15
SOME / 15 => 15?
PUSH / _ => 'cherry'
UPDATE / 'cherry' : 15? : <-2> => <-2>
PUSH / _ => 22
SOME / 22 => 22?
PUSH / _ => 'banana'
UPDATE / 'banana' : 22? : <-2> => <-2>
DUP / <-2> => <-2> : <-2>
PUSH / _ => 'cherry'
DUP / 'cherry' => 'cherry' : 'cherry'
SWAP / 'cherry' : 'cherry' => 'cherry' : 'cherry'
DIP / * => _
SWAP / 'cherry' : <-2> => <-2> : 'cherry'
DIP 1 / _ => *
MEM / 'cherry' : <-2> => True
IF / True => _
DIP / * => _
DUP / <-2> => <-2> : <-2>
DIP 1 / _ => *
DUP / 'cherry' => 'cherry' : 'cherry'
DIP / * => _
SWAP / 'cherry' : <-2> => <-2> : 'cherry'
DIP 1 / _ => *
GET / 'cherry' : <-2> => 15?
IF_NONE / 15? => 15
PUSH / _ => 5
ADD / 5 : 15 => 20
SOME / 20 => 20?
SWAP / 20? : 'cherry' => 'cherry' : 20?
UPDATE / 'cherry' : 20? : <-2> => <-2>
NIL / _ => []
PAIR / [] : <-2> => ([] * <-2>)
END %default / ([] * <-2>) => _

type,value
big_map (string nat),-2


This example is a little far-fetched but it demonstrates how you can use the different instructions available with big maps.

## Exercises

1. Write a contract that takes a pair of 2 strings as a parameter: the first string is the key of the storage big map and the second string is an element that must be added to the set contained in the value of the big map.

In [ ]:
## Write your code here

<details>
    <summary>Solution</summary>
    <p>Copy-paste the solution in the cell above to compile it!</p>
    <br />
    <pre>
parameter (pair string string) ;
storage (big_map string (set string)) ;
code {
    UNPPAIIR ;
    DIP { SWAP } ;
    PAIR ;
    DUP ;
    UNPAIR ;
    GET ;
    IF_SOME
        {
            DIG 2 ;
            PUSH bool True; 
            SWAP ;
            UPDATE ;
            SOME ;
            SWAP ;
            UNPAIR ;
            DIP { SWAP } ;
            UPDATE ;
            NIL operation ;
            PAIR
        }
        { FAIL }
} ;

RUN %default (Pair "fruits" "cherry") { Elt "fruits" { "banana" ; "apple" ; "strawberry" }} ;
    </pre>
</details>

2. Write a contract with a storage big map whose keys are pairs of strings and values are nat values, pass a pair of string and a nat value as parameter and add it to the big map after checking if the key doesn't exist.

In [ ]:
## Write your code here

<details>
    <summary>Solution</summary>
    <p>Copy-paste the solution in the cell above to compile it!</p>
    <br />
    <pre>
parameter (pair (pair string string) nat) ;
storage (big_map (pair string string) nat) ;
code {
    DUP ;
    UNPAIR ;
    CAR ;
    GET ;
    IF_NONE
        {
            UNPPAIIR ;
            DIP { SOME } ;
            UPDATE ;
            NIL operation ;
            PAIR
        }
        { FAIL }
} ;

RUN %default (Pair (Pair "Baking" "Bad") 100) {};
    </pre>
</details>